In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch 
from torch.utils.data import IterableDataset, DataLoader
from models import NextTransactionModel, TransactionsModel
from data_generators import batches_generator, transaction_features

from embedding import EmbeddingLayer
from ptls.frames import PtlsDataModule
from collections import namedtuple

In [2]:
with open('./assets/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)

NameError: name 'pickle' is not defined

In [ ]:
class PaddedBatch:
    def __init__(self, data):
        self.payload = data
        self.seq_lens = [data.shape[1]] * data.shape[0]

In [152]:
class IterDataset(IterableDataset):
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        return self.generator
    
    def collate_fn(self, x):
        return x[0]

In [153]:
path_to_dataset = '../train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [154]:
train_generator = batches_generator(dataset_train, batch_size=128, shuffle=True,
                                    device=device, is_train=True, output_format='torch')

In [155]:
d = IterDataset(train_generator)

In [156]:
dataloader = DataLoader(d, batch_size=1)

In [157]:
train_dl = PtlsDataModule(
    train_data=d)

In [158]:
b = next(iter(train_dl.train_dataloader()))

In [159]:
emb_layer = EmbeddingLayer(transaction_features, embedding_projections).cuda()

In [160]:
class PtlsEmbeddingLayer(EmbeddingLayer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, batch):
        x = super().forward(batch)
        return PaddedBatch(x)

In [161]:
ptls_emb_layer = PtlsEmbeddingLayer(transaction_features, embedding_projections).cuda()

In [162]:
from ptls.nn import RnnSeqEncoder

seq_encoder = RnnSeqEncoder(
    trx_encoder=ptls_emb_layer,
    hidden_size=256,
    type='gru',
).cuda()

In [163]:
# %debug
seq_encoder(b)

tensor([[ 0.0368, -0.0537,  0.0037,  ...,  0.1051,  0.0051, -0.0569],
        [ 0.0300, -0.0821, -0.0353,  ...,  0.0779, -0.0949,  0.0174],
        [ 0.0947, -0.0972,  0.0442,  ...,  0.0120,  0.0570, -0.0740],
        ...,
        [-0.2860,  0.2188, -0.2238,  ...,  0.0070, -0.2539,  0.0492],
        [ 0.4329,  0.5534, -0.4534,  ...,  0.1443, -0.2891, -0.3040],
        [ 0.0370, -0.0492, -0.0097,  ...,  0.1026,  0.0044, -0.0537]],
       device='cuda:0', grad_fn=<IndexBackward0>)